
#  Air Quality Forecasting - Final Training Script

# This script trains and evaluates the final champion models based on the findings from the exploratory notebook.
# It includes:
# 1. A strong XGBoost baseline model.
# 2. Two specialized, hyperparameter-tuned Temporal LSTM models (one for O3, one for NO2).





#  Cell 1: Imports and Global Setup

In [5]:

print("[DEBUG] Importing core libraries...")
import warnings
warnings.filterwarnings("ignore")
import os
import math
import time
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

print("[DEBUG] Libraries loaded.")




[DEBUG] Importing core libraries...
[DEBUG] Libraries loaded.


#  Cell 2: Global Configuration & Best Hyperparameters

In [6]:
print("[DEBUG] Setting global configuration...")
# --- Path and Data Configuration ---
BASE_DIR = Path("./PS2-SIH25").resolve()
DATA_DIR = BASE_DIR / "Data_SIH_2025 2"
ARTIFACT_DIR = Path("artifacts/final_models")
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

SITE_IDS = [f"site_{i}" for i in range(1, 8)]
TARGET_COLUMNS = ["O3_target", "NO2_target"]
FORECAST_COLUMNS = ["O3_forecast", "NO2_forecast", "T_forecast", "q_forecast", "u_forecast", "v_forecast", "w_forecast"]
SATELLITE_COLUMNS = ["NO2_satellite", "HCHO_satellite", "ratio_satellite"]
TIME_COLUMNS = ["year", "month", "day", "hour"]

# --- Model & Training Configuration ---
RANDOM_SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RESULTS = {}
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# --- Best Optuna Hyperparameters for LSTM Models ---
# NOTE: The 'horizon' is set to 48 for both models to meet the final problem statement.
BEST_LSTM_PARAMS = {
    "O3_target": {
        'window': 72,
        'horizon': 48,
        'batch_size': 128,
        'hidden_size': 64,
        'num_layers': 2,
        'dropout': 0.35,
        'lr': 0.0008753,
        'patience': 8
    },
    "NO2_target": {
        'window': 48,
        'horizon': 48,
        'batch_size': 96,
        'hidden_size': 96,
        'num_layers': 1,
        'dropout': 0.2,
        'lr': 0.0009557,
        'patience': 7
    }
}

print(f"[DEBUG] Base dir: {BASE_DIR}")
print(f"[DEBUG] Data dir: {DATA_DIR}")
print(f"[DEBUG] Artifacts will be saved to: {ARTIFACT_DIR}")
print(f"[DEBUG] Using device: {DEVICE}")

[DEBUG] Setting global configuration...
[DEBUG] Base dir: /content/PS2-SIH25
[DEBUG] Data dir: /content/PS2-SIH25/Data_SIH_2025 2
[DEBUG] Artifacts will be saved to: artifacts/final_models
[DEBUG] Using device: cuda


# Cell 3: Core Data Loading & Utility Functions

In [7]:
print("[DEBUG] Defining data utility helpers...")

def ensure_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df[["year", "month", "day", "hour"]], errors="coerce")
    return df

def load_site_dataframe(site_id: str) -> pd.DataFrame:
    path = DATA_DIR / f"{site_id}_train_data.csv"
    df = pd.read_csv(path)
    df["site_id"] = site_id
    df = ensure_timestamp(df)
    df = df.sort_values("timestamp").reset_index(drop=True)
    numeric_cols = df.select_dtypes(include=["float", "int"]).columns.tolist()
    df[numeric_cols] = df[numeric_cols].interpolate().ffill().bfill()
    return df

def load_all_train_data() -> pd.DataFrame:
    frames = [load_site_dataframe(site_id) for site_id in SITE_IDS]
    combined = pd.concat(frames, axis=0, ignore_index=True)
    combined = combined.sort_values(["timestamp", "site_id"]).reset_index(drop=True)
    print(f"[DEBUG] Loaded and combined data for {len(SITE_IDS)} sites. Total rows: {len(combined)}")
    return combined

def chronological_split(df: pd.DataFrame, split_ratio: float = 0.8):
    unique_ts = np.sort(df["timestamp"].unique())
    cutoff_index = int(len(unique_ts) * split_ratio)
    cutoff = unique_ts[cutoff_index]
    train_mask = df["timestamp"] <= cutoff
    train_df = df.loc[train_mask].reset_index(drop=True)
    val_df = df.loc[~train_mask].reset_index(drop=True)
    print(f"[DEBUG] Chronological split at {cutoff} -> train {len(train_df)}, val {len(val_df)}")
    return train_df, val_df

def evaluate_predictions(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return {"mae": float(mae), "rmse": float(rmse), "r2": float(r2)}

print("[DEBUG] Utility functions defined.")

[DEBUG] Defining data utility helpers...
[DEBUG] Utility functions defined.


# Cell 4: Initial Data Loading

In [8]:
print("[DEBUG] Loading and preparing the main dataframe...")
ALL_TRAIN_DF = load_all_train_data()
print("[DEBUG] Main dataframe ready.")


[DEBUG] Loading and preparing the main dataframe...
[DEBUG] Loaded and combined data for 7 sites. Total rows: 171679
[DEBUG] Main dataframe ready.


# Cell 5: XGBoost Model - Feature Engineering

In [9]:
print("[DEBUG] Defining feature engineering for XGBoost model...")

def add_time_signals(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["hour_sin"] = np.sin(2 * math.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * math.pi * df["hour"] / 24)
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    df["dow_sin"] = np.sin(2 * math.pi * df["dayofweek"] / 7)
    df["dow_cos"] = np.cos(2 * math.pi * df["dayofweek"] / 7)
    return df

def add_lag_features(df: pd.DataFrame, columns: list[str], window: int) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        for lag in range(1, window + 1):
            df[f"{col}_lag_{lag}"] = df.groupby("site_id")[col].shift(lag)
    return df

def build_tree_dataset(base_df: pd.DataFrame, target_col: str, horizon: int, lag_window: int) -> pd.DataFrame:
    records = []
    feature_columns_base = TIME_COLUMNS + FORECAST_COLUMNS + SATELLITE_COLUMNS
    for site_id, site_df in base_df.groupby("site_id"):
        enriched = add_time_signals(site_df)
        enriched = add_lag_features(enriched, feature_columns_base, lag_window)
        enriched["site_numeric"] = int(site_id.split("_")[1])

        feature_cols = [col for col in enriched.columns if col not in TARGET_COLUMNS + ["timestamp", "site_id"]]
        base_frame = enriched[["timestamp", "site_id"] + feature_cols].copy()

        for h in range(1, horizon + 1):
            horizon_frame = base_frame.copy()
            horizon_frame["horizon"] = h
            horizon_frame["target"] = enriched[target_col].shift(-h)
            records.append(horizon_frame)

    dataset = pd.concat(records, ignore_index=True)
    dataset = dataset.dropna(subset=["target"])
    print(f"[DEBUG] Built tree dataset for {target_col}: {dataset.shape}")
    return dataset

def prepare_tree_matrices(df: pd.DataFrame):
    feature_cols = [col for col in df.columns if col not in ["timestamp", "target", "site_id"]]
    X = df[feature_cols].values
    y = df["target"].values
    return X, y, feature_cols

print("[DEBUG] XGBoost feature functions defined.")

[DEBUG] Defining feature engineering for XGBoost model...
[DEBUG] XGBoost feature functions defined.


# Cell 6: XGBoost Model - Training, Evaluation & Saving

In [44]:
print("\\n" + "="*50)
print("### Training XGBoost Baseline Models ###")
print("="*50 + "\\n")

import pickle # <-- 1. IMPORT THE PICKLE LIBRARY

RESULTS["xgboost"] = {}
XGB_MODELS = {}

for target_col in TARGET_COLUMNS:
    print(f"--- Training XGBoost for {target_col} ---")
    dataset = build_tree_dataset(ALL_TRAIN_DF, target_col, horizon=24, lag_window=24)
    train_df, val_df = chronological_split(dataset, split_ratio=0.8)

    X_train, y_train, feature_cols = prepare_tree_matrices(train_df)
    X_val, y_val, _ = prepare_tree_matrices(val_df)

    model = XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=RANDOM_SEED,
        tree_method="hist",
        n_jobs=-1,
        early_stopping_rounds=30
    )

    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

    preds = model.predict(X_val)
    metrics = evaluate_predictions(y_val, preds)
    RESULTS["xgboost"][target_col] = metrics

    # --- CHANGE: Save a checkpoint dictionary using pickle ---

    # 2. Create a dictionary containing the model and its features
    checkpoint = {
        "model": model,
        "feature_cols": feature_cols
    }

    # 3. Change the file extension to .pkl
    model_save_path = ARTIFACT_DIR / f"xgboost_{target_col}_checkpoint.pkl"

    # 4. Save the checkpoint dictionary with pickle.dump
    with open(model_save_path, 'wb') as f:
        pickle.dump(checkpoint, f)

    print(f"[INFO] Saved XGBoost checkpoint for {target_col} to {model_save_path}")
    print(f"[INFO] XGBoost metrics for {target_col} -> MAE={metrics['mae']:.4f}, RMSE={metrics['rmse']:.4f}, R2={metrics['r2']:.4f}\\n")

# --- Calculate and store macro averages ---
macro_mae = np.mean([m['mae'] for m in RESULTS["xgboost"].values()])
macro_rmse = np.mean([m['rmse'] for m in RESULTS["xgboost"].values()])
macro_r2 = np.mean([m['r2'] for m in RESULTS["xgboost"].values()])
RESULTS["xgboost"]["macro"] = {"mae": macro_mae, "rmse": macro_rmse, "r2": macro_r2}

\n==================================================
### Training XGBoost Baseline Models ###
==================================================\n
--- Training XGBoost for O3_target ---
[DEBUG] Built tree dataset for O3_target: (4118196, 360)
[DEBUG] Chronological split at 2023-06-17T14:00:00.000000000 -> train 3289248, val 828948
[INFO] Saved XGBoost checkpoint for O3_target to artifacts/final_models/xgboost_O3_target_checkpoint.pkl
[INFO] XGBoost metrics for O3_target -> MAE=18.4717, RMSE=28.1014, R2=0.3438\n
--- Training XGBoost for NO2_target ---
[DEBUG] Built tree dataset for NO2_target: (4118196, 360)
[DEBUG] Chronological split at 2023-06-17T14:00:00.000000000 -> train 3289248, val 828948
[INFO] Saved XGBoost checkpoint for NO2_target to artifacts/final_models/xgboost_NO2_target_checkpoint.pkl
[INFO] XGBoost metrics for NO2_target -> MAE=17.4989, RMSE=25.6756, R2=0.2207\n


# Cell 7: Temporal LSTM - Feature, Model & Training Functions

In [11]:
print("[DEBUG] Defining sequence preparation and LSTM utilities...")

# --- Feature and Sequence Generation ---
def add_rolling_features(df: pd.DataFrame, columns: list[str], windows: list[int]) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        if col not in df.columns: continue
        for window in windows:
            feature_name = f"{col}_roll_mean_{window}"
            df[feature_name] = df.groupby("site_id")[col].transform(lambda s: s.shift(1).rolling(window, min_periods=1).mean())
    return df

def build_feature_matrix(df: pd.DataFrame, add_site_one_hot: bool = True) -> tuple[pd.DataFrame, list[str]]:
    base = df.copy()
    base = add_time_signals(base)
    if add_site_one_hot:
        site_dummies = pd.get_dummies(base["site_id"], prefix="site")
        base = pd.concat([base, site_dummies], axis=1)
    feature_cols = [col for col in base.columns if col not in TARGET_COLUMNS + ["timestamp", "site_id"]]
    return base, feature_cols

def generate_sequences(df: pd.DataFrame, feature_cols: list[str], window: int, horizon: int) -> list[dict]:
    sequences = []
    for site_id, site_df in df.groupby("site_id"):
        site_df = site_df.reset_index(drop=True)
        feat_vals = site_df[feature_cols].to_numpy(dtype=np.float32)
        targ_vals = site_df[TARGET_COLUMNS].to_numpy(dtype=np.float32)
        for idx in range(window, len(site_df) - horizon):
            x_window = feat_vals[idx - window: idx]
            y_window = targ_vals[idx: idx + horizon]
            if not np.all(np.isfinite(x_window)) or not np.all(np.isfinite(y_window)): continue
            sequences.append({"x": x_window, "y": y_window})
    print(f"[DEBUG] Generated {len(sequences)} sequences (window={window}, horizon={horizon})")
    return sequences

def project_sequences_to_target(sequences, target_col):
    idx = TARGET_COLUMNS.index(target_col)
    return [{**seq, 'y': seq['y'][:, [idx]]} for seq in sequences]

def split_sequences_lstm(sequences: list, train_ratio=0.7, val_ratio=0.15):
    total = len(sequences)
    train_end = int(total * train_ratio)
    val_end = train_end + int(total * val_ratio)
    return sequences[:train_end], sequences[train_end:val_end], sequences[val_end:]

# --- Scaling ---
def fit_scalers(train_sequences: list[dict]):
    train_features = np.concatenate([seq["x"] for seq in train_sequences], axis=0)
    train_targets = np.concatenate([seq["y"] for seq in train_sequences], axis=0)
    feature_scaler = StandardScaler().fit(train_features)
    target_scaler = StandardScaler().fit(train_targets)
    return feature_scaler, target_scaler

def apply_scalers(sequences: list[dict], feature_scaler: StandardScaler, target_scaler: StandardScaler):
    return [{**seq, "x": feature_scaler.transform(seq["x"]), "y": target_scaler.transform(seq["y"])} for seq in sequences]

def inverse_transform_predictions(preds: np.ndarray, scaler: StandardScaler) -> np.ndarray:
    flat = preds.reshape(-1, preds.shape[-1])
    restored = scaler.inverse_transform(flat)
    return restored.reshape(preds.shape)

# --- PyTorch Components ---
class SequenceDataset(Dataset):
    def __init__(self, sequences: list[dict]):
        self.features = torch.tensor(np.stack([seq["x"] for seq in sequences]), dtype=torch.float32)
        self.targets = torch.tensor(np.stack([seq["y"] for seq in sequences]), dtype=torch.float32)
    def __len__(self): return self.features.shape[0]
    def __getitem__(self, idx: int): return self.features[idx], self.targets[idx]

class TemporalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, horizon, target_dim, num_layers=2, dropout=0.2):
        super().__init__()
        self.horizon, self.target_dim = horizon, target_dim
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, horizon * target_dim)
    def forward(self, x):
        output, _ = self.lstm(x)
        preds = self.fc(output[:, -1, :])
        return preds.view(-1, self.horizon, self.target_dim)

def train_lstm(model, train_loader, val_loader, epochs=40, lr=1e-3, patience=5, weight_decay=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay) # Added weight_decay
    criterion = nn.MSELoss()
    best_val_loss = float("inf")
    best_state = None
    stale_epochs = 0
    for epoch in range(1, epochs + 1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                val_losses.append(criterion(model(xb), yb).item())

        val_loss = np.mean(val_losses)
        print(f"[DEBUG] Epoch {epoch:02d} | val_loss={val_loss:.5f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            stale_epochs = 0
        else:
            stale_epochs += 1
            if stale_epochs >= patience:
                print(f"[DEBUG] Early stopping triggered after {epoch} epochs.")
                break

    if best_state:
        model.load_state_dict(best_state)
    return model

print("[DEBUG] LSTM utilities defined.")


[DEBUG] Defining sequence preparation and LSTM utilities...
[DEBUG] LSTM utilities defined.


# Cell 8: Temporal LSTM - Final Training, Evaluation & Saving

In [37]:
print("\n" + "="*50)
print("### Training Tuned Temporal LSTM Models ###")
print("="*50 + "\n")

RESULTS["tuned_lstm"] = {}
FINAL_LSTM_MODELS = {}

# --- Prepare the base dataframe with all possible features once ---
lstm_base_df = add_rolling_features(ALL_TRAIN_DF, FORECAST_COLUMNS + SATELLITE_COLUMNS + TARGET_COLUMNS, [6, 24])
base_feature_df, all_lstm_feature_cols = build_feature_matrix(lstm_base_df)

for target_col in TARGET_COLUMNS:
    print(f"--- Training Tuned LSTM for {target_col} ---")
    params = BEST_LSTM_PARAMS[target_col]

    # --- Data Preparation ---
    sequence_records = generate_sequences(base_feature_df, all_lstm_feature_cols, params['window'], params['horizon'])
    projected_sequences = project_sequences_to_target(sequence_records, target_col)

    train_seq, val_seq, test_seq = split_sequences_lstm(projected_sequences)

    feature_scaler, target_scaler = fit_scalers(train_seq)

    train_scaled = apply_scalers(train_seq, feature_scaler, target_scaler)
    val_scaled = apply_scalers(val_seq, feature_scaler, target_scaler)
    test_scaled = apply_scalers(test_seq, feature_scaler, target_scaler)

    train_loader = DataLoader(SequenceDataset(train_scaled), batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(SequenceDataset(val_scaled), batch_size=params['batch_size'])
    test_loader = DataLoader(SequenceDataset(test_scaled), batch_size=params['batch_size'])

    # --- Model Training ---
    model = TemporalLSTM(
        input_size=len(all_lstm_feature_cols),
        hidden_size=params['hidden_size'],
        horizon=params['horizon'],
        target_dim=1,
        num_layers=params['num_layers'],
        dropout=params['dropout']
    ).to(DEVICE)

    model = train_lstm(model, train_loader, val_loader, epochs=40, lr=params['lr'], patience=params['patience'])

    # --- Evaluation ---
    model.eval()
    all_preds, all_targets = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            all_preds.append(model(xb.to(DEVICE)).cpu().numpy())
            all_targets.append(yb.numpy())

    pred_array = np.concatenate(all_preds)
    target_array = np.concatenate(all_targets)

    pred_restored = inverse_transform_predictions(pred_array, target_scaler)
    target_restored = inverse_transform_predictions(target_array, target_scaler)

    metrics = evaluate_predictions(target_restored.flatten(), pred_restored.flatten())
    RESULTS["tuned_lstm"][target_col] = metrics

    # --- Save the Champion LSTM Model ---
    checkpoint = {
        "model_state": model.state_dict(),
        "params": params,
        "feature_cols": all_lstm_feature_cols,  # <-- FIX: Added the missing feature list here
        "target_scaler": target_scaler,
        "feature_scaler": feature_scaler,
        "metrics": metrics,
        "timestamp_utc": datetime.utcnow().isoformat() + "Z",
    }
    model_save_path = ARTIFACT_DIR / f"lstm_{target_col}_champion.pt"
    torch.save(checkpoint, model_save_path)

    print(f"[INFO] Saved Tuned LSTM model for {target_col} to {model_save_path}")
    print(f"[INFO] Tuned LSTM metrics for {target_col} -> MAE={metrics['mae']:.4f}, RMSE={metrics['rmse']:.4f}, R2={metrics['r2']:.4f}\n")

# --- Calculate and store macro averages ---
macro_mae_lstm = np.mean([m['mae'] for m in RESULTS["tuned_lstm"].values()])
macro_rmse_lstm = np.mean([m['rmse'] for m in RESULTS["tuned_lstm"].values()])
macro_r2_lstm = np.mean([m['r2'] for m in RESULTS["tuned_lstm"].values()])
RESULTS["tuned_lstm"]["macro"] = {"mae": macro_mae_lstm, "rmse": macro_rmse_lstm, "r2": macro_r2_lstm}


### Training Tuned Temporal LSTM Models ###

--- Training Tuned LSTM for O3_target ---
[DEBUG] Generated 170832 sequences (window=72, horizon=48)
[DEBUG] Epoch 01 | val_loss=0.19943
[DEBUG] Epoch 02 | val_loss=0.21271
[DEBUG] Epoch 03 | val_loss=0.22928
[DEBUG] Epoch 04 | val_loss=0.22140
[DEBUG] Epoch 05 | val_loss=0.21308
[DEBUG] Epoch 06 | val_loss=0.22505
[DEBUG] Epoch 07 | val_loss=0.21688
[DEBUG] Epoch 08 | val_loss=0.22331
[DEBUG] Epoch 09 | val_loss=0.22237
[DEBUG] Early stopping triggered after 9 epochs.
[INFO] Saved Tuned LSTM model for O3_target to artifacts/final_models/lstm_O3_target_champion.pt
[INFO] Tuned LSTM metrics for O3_target -> MAE=14.7698, RMSE=21.7439, R2=0.6659

--- Training Tuned LSTM for NO2_target ---
[DEBUG] Generated 171000 sequences (window=48, horizon=48)
[DEBUG] Epoch 01 | val_loss=0.27028
[DEBUG] Epoch 02 | val_loss=0.31302
[DEBUG] Epoch 03 | val_loss=0.31550
[DEBUG] Epoch 04 | val_loss=0.31165
[DEBUG] Epoch 05 | val_loss=0.31100
[DEBUG] Epoch 06 | v

# Cell 9: Final Results Summary

In [13]:
print("\n" + "="*50)
print("### Final Model Comparison ###")
print("="*50 + "\n")

summary_records = []
for model_name, model_results in RESULTS.items():
    for target_name, metrics in model_results.items():
        summary_records.append({
            "model": model_name,
            "target": target_name,
            **metrics
        })

summary_df = pd.DataFrame(summary_records)
print(summary_df.to_string())

# --- Declare the final champion based on macro MAE ---
xgb_mae = RESULTS.get("xgboost", {}).get("macro", {}).get("mae", float('inf'))
lstm_mae = RESULTS.get("tuned_lstm", {}).get("macro", {}).get("mae", float('inf'))

print("\n---")
if lstm_mae < xgb_mae:
    print(f"[CHAMPION] The Tuned LSTM model is the winner with a macro MAE of {lstm_mae:.4f} (vs. XGBoost's {xgb_mae:.4f})")
else:
    print(f"[CHAMPION] The XGBoost model is the winner with a macro MAE of {xgb_mae:.4f} (vs. Tuned LSTM's {lstm_mae:.4f})")
print("---\n")


### Final Model Comparison ###

        model      target        mae       rmse        r2
0     xgboost   O3_target  18.471678  28.101396  0.343793
1     xgboost  NO2_target  17.498871  25.675592  0.220721
2     xgboost       macro  17.985274  26.888494  0.282257
3  tuned_lstm   O3_target  14.806224  21.640255  0.669037
4  tuned_lstm  NO2_target  12.410555  17.837553  0.432495
5  tuned_lstm       macro  13.608389  19.738904  0.550766

---
[CHAMPION] The Tuned LSTM model is the winner with a macro MAE of 13.6084 (vs. XGBoost's 17.9853)
---



# Cell 1: Imports and Global Setup

In [15]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import math

import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

print("[DEBUG] Libraries loaded.")

[DEBUG] Libraries loaded.


# Cell 2: Configuration - Point to Models and Data

In [16]:
print("[DEBUG] Setting configuration...")

# --- Configure paths ---
# Ensure these paths match where your models and data are stored
BASE_DIR = Path("./PS2-SIH25").resolve()
DATA_DIR = BASE_DIR / "Data_SIH_2025 2"
ARTIFACT_DIR = Path("artifacts/final_models")

# --- Select a site to test ---
SITE_ID_TO_TEST = "site_1"

# --- Define column groups (should match your training script) ---
TARGET_COLUMNS = ["O3_target", "NO2_target"]
FORECAST_COLUMNS = ["O3_forecast", "NO2_forecast", "T_forecast", "q_forecast", "u_forecast", "v_forecast", "w_forecast"]
SATELLITE_COLUMNS = ["NO2_satellite", "HCHO_satellite", "ratio_satellite"]
TIME_COLUMNS = ["year", "month", "day", "hour"]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"[INFO] Running test case for site: {SITE_ID_TO_TEST}")
print(f"[INFO] Using device: {DEVICE}")



[DEBUG] Setting configuration...
[INFO] Running test case for site: site_1
[INFO] Using device: cuda


# Cell 3: Utility Functions & Model Class Definition


In [17]:
print("[DEBUG] Defining utility functions and LSTM class...")

def ensure_timestamp(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df[["year", "month", "day", "hour"]], errors="coerce")
    return df

def load_dataframe(site_id: str, suffix: str) -> pd.DataFrame:
    path = DATA_DIR / f"{site_id}{suffix}"
    df = pd.read_csv(path)
    df["site_id"] = site_id
    df = ensure_timestamp(df)
    df = df.sort_values("timestamp").reset_index(drop=True)
    numeric_cols = df.select_dtypes(include=["float", "int"]).columns.tolist()
    df[numeric_cols] = df[numeric_cols].interpolate().ffill().bfill()
    return df

def add_time_signals(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["hour_sin"] = np.sin(2 * math.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * math.pi * df["hour"] / 24)
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    df["dow_sin"] = np.sin(2 * math.pi * df["dayofweek"] / 7)
    df["dow_cos"] = np.cos(2 * math.pi * df["dayofweek"] / 7)
    return df

def add_lag_features(df: pd.DataFrame, columns: list[str], window: int) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        for lag in range(1, window + 1):
            df[f"{col}_lag_{lag}"] = df.groupby("site_id")[col].shift(lag)
    return df

def add_rolling_features(df: pd.DataFrame, columns: list[str], windows: list[int]) -> pd.DataFrame:
    df = df.copy()
    for col in columns:
        if col not in df.columns: continue
        for window in windows:
            feature_name = f"{col}_roll_mean_{window}"
            df[feature_name] = df.groupby("site_id")[col].transform(lambda s: s.shift(1).rolling(window, min_periods=1).mean())
    return df

# Define the LSTM model class EXACTLY as it was during training
class TemporalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, horizon, target_dim, num_layers=2, dropout=0.2):
        super().__init__()
        self.horizon, self.target_dim = horizon, target_dim
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, horizon * target_dim)
    def forward(self, x):
        output, _ = self.lstm(x)
        preds = self.fc(output[:, -1, :])
        return preds.view(-1, self.horizon, self.target_dim)

print("[DEBUG] Utilities ready.")




[DEBUG] Defining utility functions and LSTM class...
[DEBUG] Utilities ready.


# Cell 4: Load Historical and Unseen Data

In [18]:
# Load the tail of the training data and all of the unseen data
train_df = load_dataframe(SITE_ID_TO_TEST, "_train_data.csv")
unseen_df = load_dataframe(SITE_ID_TO_TEST, "_unseen_input_data.csv")

# We need enough history for the longest lookback window (72 for O3 LSTM)
# We'll take the last 200 hours of training data as a safe buffer
historical_context_df = train_df.tail(200)

# The input data for prediction will be this historical context followed by the unseen data
input_df = pd.concat([historical_context_df, unseen_df], ignore_index=True)

print(f"[INFO] Prepared input data of shape: {input_df.shape}")
print(f"[INFO] Forecasting from the last known timestamp: {unseen_df['timestamp'].max()}")



[INFO] Prepared input data of shape: (11072, 18)
[INFO] Forecasting from the last known timestamp: 2024-06-29 23:00:00


# Cell 5: Test Case for XGBoost Models

In [47]:
print("\n" + "="*50)
print("### Testing XGBoost Models (with Pickle) ###")
print("="*50 + "\n")

import pickle

try:
    # --- Prepare data as before ---
    xgb_input_df = input_df.copy()
    rolling_feature_cols = FORECAST_COLUMNS + SATELLITE_COLUMNS + TARGET_COLUMNS
    xgb_input_df = add_rolling_features(xgb_input_df, rolling_feature_cols, [6, 24])
    xgb_input_df = add_time_signals(xgb_input_df)

    # --- FIX: Added TIME_COLUMNS to the list of features to be lagged ---
    lag_feature_cols = TIME_COLUMNS + FORECAST_COLUMNS + SATELLITE_COLUMNS

    xgb_input_df = add_lag_features(xgb_input_df, lag_feature_cols, 24)
    xgb_input_df["site_numeric"] = int(SITE_ID_TO_TEST.split("_")[1])

    prediction_row = xgb_input_df.dropna().tail(1)

    if prediction_row.empty:
        print("\n[FATAL ERROR] The dataframe is empty after dropping NaNs.")
    else:
        for target_col in TARGET_COLUMNS:
            print(f"\n--- Loading and predicting for {target_col} with XGBoost ---")

            # --- Load Checkpoint from Pickle File ---
            checkpoint_path = ARTIFACT_DIR / f"xgboost_{target_col}_checkpoint.pkl"
            with open(checkpoint_path, 'rb') as f:
                checkpoint = pickle.load(f)

            model = checkpoint["model"]
            feature_names_from_training = checkpoint["feature_cols"]
            print(f"[INFO] Loaded checkpoint for {target_col} from {checkpoint_path}")

            # --- Make Prediction ---
            prediction_df = pd.concat([prediction_row] * 24, ignore_index=True)
            prediction_df['horizon'] = np.arange(1, 25)

            X_test = prediction_df[feature_names_from_training]

            predictions = model.predict(X_test)

            print(f"[XGBoost] 24-Hour Forecast for {target_col}:")
            for i in range(24):
                print(f"  - Hour T+{i+1}: {predictions[i]:.4f}")
            print("-" * 20)

except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")


### Testing XGBoost Models (with Pickle) ###


--- Loading and predicting for O3_target with XGBoost ---
[INFO] Loaded checkpoint for O3_target from artifacts/final_models/xgboost_O3_target_checkpoint.pkl
[XGBoost] 24-Hour Forecast for O3_target:
  - Hour T+1: -4.4780
  - Hour T+2: -4.4373
  - Hour T+3: -2.3704
  - Hour T+4: 7.6461
  - Hour T+5: 19.7956
  - Hour T+6: 34.0199
  - Hour T+7: 37.3157
  - Hour T+8: 41.6491
  - Hour T+9: 41.6381
  - Hour T+10: 41.9554
  - Hour T+11: 37.6496
  - Hour T+12: 29.8153
  - Hour T+13: 21.7244
  - Hour T+14: 17.8686
  - Hour T+15: 10.3316
  - Hour T+16: 3.7187
  - Hour T+17: 2.5342
  - Hour T+18: 0.1007
  - Hour T+19: -2.1686
  - Hour T+20: -6.0530
  - Hour T+21: -8.4266
  - Hour T+22: -9.5110
  - Hour T+23: -10.4798
  - Hour T+24: -10.1351
--------------------

--- Loading and predicting for NO2_target with XGBoost ---
[INFO] Loaded checkpoint for NO2_target from artifacts/final_models/xgboost_NO2_target_checkpoint.pkl
[XGBoost] 24-Hour Forecast f

# Cell 6: Test Case for Tuned LSTM Models

In [43]:
print("\n" + "="*50)
print("### Testing Tuned LSTM Models ###")
print("="*50 + "\n")

for target_col in TARGET_COLUMNS:
    print(f"--- Loading and predicting for {target_col} with LSTM ---")

    # --- Load Checkpoint ---
    checkpoint_path = ARTIFACT_DIR / f"lstm_{target_col}_champion.pt"
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE, weights_only=False)

    # --- Recreate Model from Checkpoint ---
    params = checkpoint['params']
    model = TemporalLSTM(
        input_size=len(checkpoint['feature_cols']),
        hidden_size=params['hidden_size'],
        horizon=params['horizon'],
        target_dim=1,
        num_layers=params['num_layers'],
        dropout=params['dropout']
    ).to(DEVICE)
    model.load_state_dict(checkpoint['model_state'])
    model.eval()

    # --- Load Scalers ---
    feature_scaler = checkpoint['feature_scaler']
    target_scaler = checkpoint['target_scaler']

    # --- Prepare data specifically for this LSTM ---
    lstm_input_df = add_rolling_features(input_df, FORECAST_COLUMNS + SATELLITE_COLUMNS + TARGET_COLUMNS, [6, 24])
    lstm_feature_df = add_time_signals(lstm_input_df)

    site_dummies = pd.get_dummies(lstm_feature_df["site_id"], prefix="site")
    lstm_feature_df = pd.concat([lstm_feature_df, site_dummies], axis=1)

    inference_window = lstm_feature_df.tail(params['window'])

    X_test_raw = inference_window.reindex(columns=checkpoint['feature_cols'], fill_value=0)

    X_test_scaled = feature_scaler.transform(X_test_raw)

    # --- FIX: Add a check for NaNs and fill them ---
    if np.isnan(X_test_scaled).any():
        print("[WARNING] NaNs found in the scaled input data! This is likely the cause of 'nan' predictions.")
        print("[INFO] Attempting to fix by filling NaNs with 0 (mean value for StandardScaler)...")
        X_test_scaled = np.nan_to_num(X_test_scaled)

    # Convert to tensor and add batch dimension
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(0).to(DEVICE)

    # --- Make Prediction ---
    with torch.no_grad():
        prediction_scaled = model(X_test_tensor)

    # Inverse transform the prediction to get the real value
    prediction_restored = target_scaler.inverse_transform(prediction_scaled.cpu().numpy().reshape(-1, 1))

    print(f"[LSTM] 48-Hour Forecast for {target_col}:")
    for i in range(params['horizon']):
        print(f"  - Hour T+{i+1}: {prediction_restored[i][0]:.4f}")
    print("-" * 20)


### Testing Tuned LSTM Models ###

--- Loading and predicting for O3_target with LSTM ---
[WARNING] NaNs found in the scaled input data! This is likely the cause of 'nan' predictions.
[INFO] Attempting to fix by filling NaNs with 0 (mean value for StandardScaler)...
[LSTM] 48-Hour Forecast for O3_target:
  - Hour T+1: 13.8075
  - Hour T+2: 14.8228
  - Hour T+3: 14.7176
  - Hour T+4: 16.3125
  - Hour T+5: 16.4259
  - Hour T+6: 22.3516
  - Hour T+7: 26.9032
  - Hour T+8: 32.3826
  - Hour T+9: 37.8943
  - Hour T+10: 39.6267
  - Hour T+11: 39.1313
  - Hour T+12: 38.5669
  - Hour T+13: 32.4675
  - Hour T+14: 28.4669
  - Hour T+15: 23.0522
  - Hour T+16: 20.6748
  - Hour T+17: 18.4595
  - Hour T+18: 17.6999
  - Hour T+19: 16.6041
  - Hour T+20: 14.8703
  - Hour T+21: 12.9130
  - Hour T+22: 15.3206
  - Hour T+23: 12.5122
  - Hour T+24: 13.1581
  - Hour T+25: 15.2290
  - Hour T+26: 13.3412
  - Hour T+27: 14.8522
  - Hour T+28: 16.5999
  - Hour T+29: 19.6744
  - Hour T+30: 21.2216
  - Hour T+3